In [70]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import csv
import os

# Hyperparameters

In [71]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.0
torch.manual_seed(1337)

In [72]:
IS_TO_TOKEN = '#'
AS_TOKEN = '$'
END_TOKEN = '@'

# Dataset

In [73]:
DATA: dict[str, list[str]] = {
    "singular_to_plural": [],
    "present_tense_to_past_tense": [],
    "base_case_to_third_person_singular": [],
    "singular_possessive_to_plural_possessive": [],
    "comparative_adjective_to_superlative_adjective": [],
    "verb_to_progressive_verb": [],
    "verb_to_derived_agentive": [],
    "base_case_to_diminuitive": [],
}

TRAIN_DATA: dict[str, list[str]] = {
    "singular_to_plural": [],
    "present_tense_to_past_tense": [],
    "base_case_to_third_person_singular": [],
    "singular_possessive_to_plural_possessive": [],
}

VAL_DATA: dict[str, list[str]] = {
    "singular_to_plural": [],
    "present_tense_to_past_tense": [],
    "base_case_to_third_person_singular": [],
    "singular_possessive_to_plural_possessive": [],
    "comparative_adjective_to_superlative_adjective": [],
    "verb_to_progressive_verb": [],
    "verb_to_derived_agentive": [],
    "base_case_to_diminuitive": [],
}

## Download data

In [74]:
csv_filepaths = [category + ".csv" for category in DATA.keys()]
for csv_filepath in csv_filepaths:
  url = f"https://raw.githubusercontent.com/adamoosya/182Proj/main/data/{csv_filepath}"
  !wget --no-cache --backups=1 {url}

--2025-04-23 02:32:00--  https://raw.githubusercontent.com/adamoosya/182Proj/main/data/singular_to_plural.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147270 (144K) [text/plain]
Saving to: ‘singular_to_plural.csv’

singular_to_plural. 100%[===================>] 143.82K  --.-KB/s    in 0.003s  

2025-04-23 02:32:01 (51.8 MB/s) - ‘singular_to_plural.csv’ saved [147270/147270]

--2025-04-23 02:32:01--  https://raw.githubusercontent.com/adamoosya/182Proj/main/data/present_tense_to_past_tense.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response...

In [75]:
for category in DATA.keys():
  csv_filepath = category + ".csv"
  with open(csv_filepath, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    word_pairs = set()
    for row in reader:
      if len(row) >= 2:  # Ensure there are at least two elements in the row
        word1, word2 = row[0], row[1]
        if all(c in stoi for c in word1) and all(c in stoi for c in word2):
          word_pairs.add((word1, word2))
    word_pairs = list(word_pairs)
    DATA[category] = word_pairs

In [76]:
train_categories = ["singular_to_plural", "present_tense_to_past_tense", "base_case_to_third_person_singular", "singular_possessive_to_plural_possessive"]

for category, word_pairs in DATA.items():
  if category in train_categories:
    split_index = int(0.9 * len(word_pairs))
    TRAIN_DATA[category] = word_pairs[:split_index]
    VAL_DATA[category] = word_pairs[split_index:]
  else:
    VAL_DATA[category] = word_pairs

In [77]:
for category, word_pairs in DATA.items():
  print(f"{category}: {len(word_pairs)}")

singular_to_plural: 878
present_tense_to_past_tense: 618
base_case_to_third_person_singular: 674
singular_possessive_to_plural_possessive: 969
comparative_adjective_to_superlative_adjective: 1096
verb_to_progressive_verb: 640
verb_to_derived_agentive: 887
base_case_to_diminuitive: 2170


In [78]:
stoi = {
    ' ': 0,
    'a': 1,
    'b': 2,
    'c': 3,
    'd': 4,
    'e': 5,
    'f': 6,
    'g': 7,
    'h': 8,
    'i': 9,
    'j': 10,
    'k': 11,
    'l': 12,
    'm': 13,
    'n': 14,
    'o': 15,
    'p': 16,
    'q': 17,
    'r': 18,
    's': 19,
    't': 20,
    'u': 21,
    'v': 22,
    'w': 23,
    'x': 24,
    'y': 25,
    'z': 26,
    '#': 27,
    '$': 28,
    '@': 29,
    '\'': 30
}
# '# = is to'
# '$ = as'
# '@ = end token'
# '% = padding at the end to make it block size length'
vocab_size = len(stoi)

itos = {v: k for k, v in stoi.items()}

# def tokenize(text):
#     return [CHAR_TO_TOKEN[c] for c in text]

# def detokenize(tokens):
#     return ''.join([TOKEN_TO_CHAR[t] for t in tokens])

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [79]:
def get_context_test(dataset, category=None):
    '''
    For example:
    context = fox$foxes#dog$
    target = dogs
    '''
    if category is None:
        category = np.random.choice(list(dataset.keys()))
    data = dataset[category]
    n = len(data)
    s1, p1 = data[np.random.randint(n)]
    s2, p2 = data[np.random.randint(n)]
    context = s1 + IS_TO_TOKEN + p1 + AS_TOKEN + s2 + IS_TO_TOKEN
    target = s2
    return context, target

def get_context_example(dataset, category=None):
    '''
    Data is of the form:
    singular $ plural # singular $ part of a plural

    For example:
    x = fox$foxes#dog$do
    y = ox$foxes#dog$dog
    '''
    if category is None:
        category = np.random.choice(list(dataset.keys()))
    data = dataset[category]
    n = len(data)
    s1, p1 = data[np.random.randint(n)]
    s2, p2 = data[np.random.randint(n)]
    window = np.random.randint(len(p2)+1)
    p2 += END_TOKEN
    x = s1 + IS_TO_TOKEN + p1 + AS_TOKEN + s2 + IS_TO_TOKEN + p2[:window]
    y = s1[1:] + IS_TO_TOKEN + p1 + AS_TOKEN + s2 + IS_TO_TOKEN + p2[:window + 1]
    x = torch.tensor(encode(x))
    y = torch.tensor(encode(y))
    return x, y

def get_batch(dataset, category=None, batch_size=batch_size):
    x = torch.zeros((batch_size, block_size), dtype=int)
    y = torch.zeros((batch_size, block_size), dtype=int)
    mask = torch.zeros((batch_size, block_size), dtype=torch.bool)
    for i in range(batch_size):
        xi, yi = get_context_example(dataset, category=category)
        x[i, :len(xi)] = xi  # Assign values up to the original length
        y[i, :len(yi)] = yi  # The rest will be padded with 0s
        mask[i, :len(xi)] = True  # Set True for actual data tokens

    x, y, mask = x.to(device), y.to(device), mask.to(device)
    return x, y, mask

In [80]:
example = get_context_example(TRAIN_DATA)
print(decode(example[0].tolist()))
print(decode(example[1].tolist()))

mixer#mixers$gymnasium#gymnasi
ixer#mixers$gymnasium#gymnasiu


In [81]:
@torch.no_grad()
def estimate_loss(model, dataset, category=None):
    model.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        X, Y, mask = get_batch(dataset, category=category)
        logits, loss = model(X, Y, mask)
        losses[k] = loss.item()
    out = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        if mask is not None:
            # Apply the mask before the softmax
            wei = wei.masked_fill(~mask[:, None, :], float('-inf'))

        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        #out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = torch.cat([h(x, mask=mask) for h in self.heads], dim=-1)  # Pass mask

        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, mask=None):
        x = x + self.sa(self.ln1(x), mask=mask)
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, mask=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        for block in self.blocks:
          x = block(x, mask=mask)

        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def test(self, context, target):
        model.eval()
        idx = torch.tensor(encode(context)).to(device).unsqueeze(0)
        t = len(target)
        for i in range(t):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        prediction = decode(idx[0].tolist())[-t:]
        model.train()
        return prediction == target

In [82]:
def evaluate(model, dataset, category=None, num_to_evaluate=100):
    correct = 0
    total = 0
    for _ in range(num_to_evaluate):
        context, target = get_context_test(dataset, category=category)
        if model.test(context, target):
            correct += 1
        total += 1
    return correct / total

In [84]:
model = GPTLanguageModel()
model = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

losses = {
    'train': {
        'singular_to_plural': [],
        'present_tense_to_past_tense': [],
        'base_case_to_third_person_singular': [],
        'singular_possessive_to_plural_possessive': [],
    },
    'val': {
        "singular_to_plural": [],
        "present_tense_to_past_tense": [],
        "base_case_to_third_person_singular": [],
        "singular_possessive_to_plural_possessive": [],
        "comparative_adjective_to_superlative_adjective": [],
        "verb_to_progressive_verb": [],
        "verb_to_derived_agentive": [],
        "base_case_to_diminuitive": [],
    }
}
correctness = {
    'train': {
        'singular_to_plural': [],
        'present_tense_to_past_tense': [],
        'base_case_to_third_person_singular': [],
        'singular_possessive_to_plural_possessive': [],
    },
    'val': {
        "singular_to_plural": [],
        "present_tense_to_past_tense": [],
        "base_case_to_third_person_singular": [],
        "singular_possessive_to_plural_possessive": [],
        "comparative_adjective_to_superlative_adjective": [],
        "verb_to_progressive_verb": [],
        "verb_to_derived_agentive": [],
        "base_case_to_diminuitive": [],
    }
}

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        for dataset, dataset_name in zip([TRAIN_DATA, VAL_DATA], ['train', 'val']):
          loss = estimate_loss(model, dataset, category=None)
          print(f"{iter}: {dataset_name}: {loss.item():.4f}")
          score = evaluate(model, dataset, category=None)
          print(f"{iter}: {dataset_name}: {score:.4f}")

        # for dataset, dataset_name in zip([TRAIN_DATA, VAL_DATA], ['train', 'val']):
        #     for category in dataset.keys():
        #         loss = estimate_loss(model, dataset, category=category)
        #         print(f"{iter}: {dataset_name} {category}: {loss.item():.4f}")
        #         losses[dataset_name][category].append(loss)

        #         score = evaluate(model, dataset, category=category)
        #         correctness[dataset_name][category].append(score)
        #         print(f"{iter}: {dataset_name} {category}: {score:.4f}")
        #losses = estimate_loss()

    # sample a batch of data
    xb, yb, mask = get_batch(TRAIN_DATA)

    # evaluate the loss
    logits, loss = model(xb, yb, mask)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



10.713631 M parameters
0: train: 3.3023
0: train: 0.0000
0: val: 3.2857
0: val: 0.0000
500: train: 0.1711
500: train: 0.6100
500: val: 0.3443
500: val: 0.5800
1000: train: 0.1192
1000: train: 0.5400
1000: val: 0.3741
1000: val: 0.6500
1500: train: 0.1136
1500: train: 0.6400
1500: val: 0.3862
1500: val: 0.6600
2000: train: 0.1052
2000: train: 0.6400
2000: val: 0.4131
2000: val: 0.6300
2500: train: 0.1039
2500: train: 0.6300
2500: val: 0.4166
2500: val: 0.6200
3000: train: 0.1035
3000: train: 0.6500
3000: val: 0.4311
3000: val: 0.6000
3500: train: 0.1028
3500: train: 0.6200
3500: val: 0.4341
3500: val: 0.6300
4000: train: 0.1005
4000: train: 0.6300
4000: val: 0.4465
4000: val: 0.7900
4500: train: 0.1000
4500: train: 0.5900
4500: val: 0.4459
4500: val: 0.6600
4999: train: 0.1009
4999: train: 0.5000
4999: val: 0.4418
4999: val: 0.5900


In [85]:
torch.save(model.state_dict(), 'train_weights.pth')